In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
from scipy.io import loadmat, savemat
from scipy.stats import binom

In [ ]:
# batch_size = 

In [10]:
K = 2
M = [30, 35]
N = [10, 15]
batch_size = 35
hidden_layer_size = 1

In [11]:
trainY

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1,

In [40]:
def accuracy(y_true, y_pred):
    return 1 - ((np.round(y_pred) - y_true)**2).sum() / len(y_true)

In [41]:
synth_data = loadmat('./synth_dataset.mat')
synth_data
testX = synth_data['test_X'].reshape(-1, sum(m*n for m, n in zip(M, N)), 1)
testY = np.ravel(synth_data['test_Y']).reshape(-1, 1)
trainX = synth_data['train_X']
trainY = np.ravel(synth_data['train_Y'])


shuffle_idx = np.random.permutation(np.arange(len(trainX)))
trainX = trainX[shuffle_idx]
trainY = trainY[shuffle_idx]
validX = trainX[:len(trainX)//4].reshape(-1, sum(m*n for m, n in zip(M, N)), 1)
validY = trainY[:len(trainY)//4].reshape(-1, 1)
trainX = trainX[len(trainX)//4:].reshape(-1, sum(m*n for m, n in zip(M, N)), 1)
trainY = trainY[len(trainY)//4:].reshape(-1, 1)

In [44]:
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, sum(m*n for m,n in zip(M, N)), 1))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 1))
    tf_valid_dataset = tf.constant(validX, tf.float32)
    tf_valid_labels = tf.constant(validY.reshape(-1,1), tf.float32)
    tf_test_dataset = tf.constant(testX, tf.float32)
    
    layer0_weights = []
    layer0_biases = []
    for k in range(K):
        kind_weights = tf.Variable(tf.truncated_normal([M[k], 1, hidden_layer_size], stddev=0.2))
        layer0_weights.append(kind_weights)
        
        kind_biases = tf.Variable(tf.zeros([hidden_layer_size]))
        layer0_biases.append(kind_biases)
        
    layer1_weights = tf.Variable(tf.truncated_normal([sum(N[k]*hidden_layer_size for k in range(K)), 1], stddev=0.2)) 
    layer1_biases = tf.Variable(tf.zeros([1]))
    
    def model(data):
        hidden_list = []
        for k in range(K):
            idx = sum(N[_k]*M[_k] for _k in range(0, k))
            next_idx = N[k]*M[k] + idx
            kind_conv = tf.nn.conv1d(data[:, idx:next_idx], layer0_weights[k], M[k], padding='VALID')
            kind_hidden = kind_conv + layer0_biases[k]
            hidden_list.append(kind_hidden)

        hidden = tf.reshape(tf.concat(hidden_list, 1), (-1, sum(N[k]*hidden_layer_size for k in range(K))))
        logits = tf.matmul( hidden, layer1_weights) + layer1_biases
        return logits
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(.1).minimize(loss)
    
    train_prediction = tf.nn.sigmoid(logits)
    valid_prediction = tf.nn.sigmoid(model(tf_valid_dataset))
    test_prediction = tf.nn.sigmoid(model(tf_test_dataset))
    
    train_accuracy = 1 - tf.reduce_mean(tf.square((tf.round(train_prediction) - tf_train_labels)))
    valid_accuracy = 1 - tf.reduce_mean(tf.square((tf.round(valid_prediction) - tf_valid_labels)))
#     test_accuracy = tf.reduce_sum(tf.square((tf.round(test_prediction) - tf)))    


In [46]:
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (trainY.shape[0] - batch_size)
        batch_data = trainX[offset:(offset+batch_size)]
        batch_labels = trainY[offset:(offset+batch_size)]
        feed_dict = {tf_train_dataset: batch_data, 
                     tf_train_labels: batch_labels}
        _, l, batch_pred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if step % 200 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(batch_labels, batch_pred))
            print('Validation accuracy: %.1f%%' % valid_accuracy.eval())
    print('Test accuracy: %.1f%%' % accuracy(testY,test_prediction.eval()))            

Minibatch loss at step 0: 1.757577
Minibatch accuracy: 0.3%
Validation accuracy: 0.5%
Minibatch loss at step 200: 0.486929
Minibatch accuracy: 0.7%
Validation accuracy: 0.7%
Minibatch loss at step 400: 0.341991
Minibatch accuracy: 0.9%
Validation accuracy: 0.7%
Minibatch loss at step 600: 0.357381
Minibatch accuracy: 0.8%
Validation accuracy: 0.7%
Minibatch loss at step 800: 0.275935
Minibatch accuracy: 0.9%
Validation accuracy: 0.8%
Minibatch loss at step 1000: 0.299679
Minibatch accuracy: 0.8%
Validation accuracy: 0.8%
Minibatch loss at step 1200: 0.134272
Minibatch accuracy: 1.0%
Validation accuracy: 0.8%
Minibatch loss at step 1400: 0.212395
Minibatch accuracy: 0.9%
Validation accuracy: 0.8%
Minibatch loss at step 1600: 0.197030
Minibatch accuracy: 0.9%
Validation accuracy: 0.8%
Minibatch loss at step 1800: 0.199588
Minibatch accuracy: 0.9%
Validation accuracy: 0.8%
Minibatch loss at step 2000: 0.108071
Minibatch accuracy: 1.0%
Validation accuracy: 0.8%
Minibatch loss at step 2200: